In [ ]:
from bs4 import BeautifulSoup
import urllib3
http = urllib3.PoolManager()

movieListUrl = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=430156241533f1d058c603178cc3ca0e&openStartDt=2016&itemPerPage=20" 
response = http.request('GET', movieListUrl)
movieIdData = BeautifulSoup(response.data, "html.parser")

print(movieIdData.prettify())

In [2]:
import json
# 문자열 BeautifulSoup -> 파싱 결과 문자열
# batter는 리스트를 가지고 있으므로 index로 속성들에 접근한다.
# dictionary는 키로 접근
json_data2 = """
{
"id":"0001",
"type":"donut",
"batters":{
    "batter":[
        {"id":"1001", "type": "Regular"},
        {"id":"1002", "type": "Chocolate"}
    ]
    }
}"""
    
result = json.loads(json_data2);
print(result);
print(result["id"]);
print(result["batters"])
print(result["batters"]["batter"])

{'id': '0001', 'type': 'donut', 'batters': {'batter': [{'id': '1001', 'type': 'Regular'}, {'id': '1002', 'type': 'Chocolate'}]}}
0001
{'batter': [{'id': '1001', 'type': 'Regular'}, {'id': '1002', 'type': 'Chocolate'}]}
[{'id': '1001', 'type': 'Regular'}, {'id': '1002', 'type': 'Chocolate'}]


In [3]:
movieIdData = json.loads(str(movieIdData))
#print(movieIdData["movieListResult"])

In [5]:
# 무비 리스트의 갯수
# 무비 리스트 출력
# 영화 제목만 출력
print(type(movieIdData))
dataset = movieIdData['movieListResult']['movieList']
#print("총",len(dataset), "개")
#print 
#for i in dataset:
#    print(i["openDt"], i["nationAlt"], i["repGenreNm"], i["movieNm"], i["movieNmEn"])
for i in range(len(movieIdData['movieListResult']['movieList'])):
    print(movieIdData['movieListResult']['movieList'][i]["movieNm"])

<class 'dict'>
램페이지3: 백악관 심판의 날
킬링 군터
공작
너와 극장에서
식물도감
훔쳐보는 쉐어 하우스
천사의 시간
애인 : 아내의 남자
파라독스
튼튼이의 모험
이름없는 새
유니콘 원정대: 비밀의 다이어리
아직 끝나지 않았다
아일라
아이 캔 온리 이매진
배틀필드: 카르발라 전투
배틀크릭
미드나잇 선
메이플쏘프
도촬 여대생 하숙집


In [6]:
import pymysql

conn = pymysql.connect(host='192.168.0.164', port=3306, user = 'root', password='gksrlgns', use_unicode = True, db = "acorn", charset='utf8', autocommit=True )
cursor = conn.cursor()
sql = "kobis_cinema_insert"
dateStr = ""

for data in dataset:
    dateStr = "{0}-{1}-{2}".format(data["openDt"][0:4], data["openDt"][4:6],data["openDt"][6:8])
    for director in range(len(data["directors"])):
        cursor.callproc(sql, (data["movieNm"], data["movieNmEn"], data["openDt"], data["openDt"][:4], data["openDt"][4:6], data["openDt"][6:], data["repNationNm"], data["genreAlt"], data["directors"][director]["peopleNm"], 0))


In [ ]:
from bs4 import BeautifulSoup
import urllib3
import json
import pymysql

http = urllib3.PoolManager()
movieListUrl = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=430156241533f1d058c603178cc3ca0e&openStartDt=2016&itemPerPage=4000" 
response = http.request('GET', movieListUrl)

conn = pymysql.connect(host='192.168.0.164', port=3306, user = 'root', password='gksrlgns', use_unicode = True, db = "acorn", charset='utf8', autocommit=True )
cursor = conn.cursor()

movieIdData = json.loads(str(movieIdData))
movieIdData = BeautifulSoup(response.data, "html.parser")
dataset = movieIdData['movieListResult']['movieList']
counts = len(dataset)
for i in range(1, counts):
    director = movieIdData["movieListResult"]["movieList"][i]["directors"]
    nam = ""
    for name in director:
        name += name["peopleNm"] + ','
    args = (movieIdData["movieListResult"]["movieList"][i]["movieNm"],
           movieIdData["movieListResult"]["movieList"][i]["movieNmEn"],
           movieIdData["movieListResult"]["movieList"][i]["openDt"],
           movieIdData["movieListResult"]["movieList"][i]["openDt"][:4],
           movieIdData["movieListResult"]["movieList"][i]["openDt"][4:6],
           movieIdData["movieListResult"]["movieList"][i]["openDt"][6:],
           movieIdData["movieListResult"]["movieList"][i]["repNationNm"],
           movieIdData["movieListResult"]["movieList"][i]["genreAlt"],
           str(nam), 0)
    cur.callproc("kobis_cinema_insert", args)
    result = cur.rowcount
print(result)

In [ ]:
cur.callproc("kobis_cinema_select")
if cur.rowcount:
    result = list(cur.fetchall())
for data in result:
    print(data)



In [ ]:
from bs4 import BeautifulSoup
from lxml import html
import re
import requests
import urllib.request
import urllib3
import json
import pymysql

http = urllib3.PoolManager()
conn = pymysql.connect(host='192.168.0.164', port=3306, user = 'root', password='gksrlgns', use_unicode = True, db = "acorn", charset='utf8', autocommit=True )
cur = conn.cursor()

def total_movie():
    score = []
    review = []
    user = []
    movie = []
    
page = 5

req = urllib.request
d = req.urlopen("https://movie.naver.com/movie/point/af/list.nhn")
bs = BeautifulSoup(d.read(), "html.parser")
body = bs.body
home = body.find_all('option')

strHome = str(home)
s = 0
while 'value' in strHome[s:]:
    s = strHome.find('value', s + 1, len(strHome))
    
    for i in range(1, len(home)):
        if home[i]['value'].isdigit() == False:
            break

        start = int(home[i]['value'])
        end = start + page

        for t in range(start, end):
            url = 'http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=%s&target=after&page=' % t 
            response = requests.get(url)
            tree = html.fromstring(response.text)
            num = tree.xpath('//*[@id="old_content"]/h5/div/strong/text()')

            if len(num) == 0 :
                continue
            elif int(num[0]) < 50:
                continue
            else:
                tot_num = (int(num[0]) // 10) + 1
                if tot_num > 10:
                    tot_num = 10

            for n in range(1, tot_num + 1):
                url = 'http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=%s&target=after&page=' % t + str(n)
                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'html.parser')
                tr_num = soup.find_all('tbody')[0]
                tr_num2 = len(tr_num.find_all('tr'))
                for i in range(1, tr_num2+1):
                    tree = html.fromstring(response.text)
                    score = tree.xpath('//*[@id="old_content"]/table/tbody/tr[%s]/td[3]/text()' % (i))
                    score = int(score[0])
                    if score < 5:
                        continue

                    review = tree.xpath('//*[@id="old_content"]/table/tbody/tr[%s]/td[4]/text()' %(i))
                    review = re.sub("\r|\n|\r", "", review[2])

                    id_name = tree.xpath('//*[@id="old_content"]/table/tbody/tr[%s]/td[5]/a/text()' % (i))
                    user = id_name

                    movie_name = tree.xpath('//*[@id="old_content"]/table/tbody/tr[%s]/td[4]/a[1]/text()' % (i))
                    movie = movie_name

                    sql = "INSERT INTO moviedb(score, review, user, movie) VALUES (%s, %s, %s, %s)"
                    cur.execute(sql, (score, review, user, movie))
                    conn.commit()



In [1]:
from bs4 import BeautifulSoup
import urllib3
http = urllib3.PoolManager()

url = "https://www.kpx.or.kr/www/contents.do?key=82#n"
response = http.request('GET', url)
sData = BeautifulSoup(response.data, "html.parser")

print(sData.prettify())

C:\Users\acorn\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<!DOCTYPE html>
<html lang="ko">
 <head>
  <title>
   발전소위치 &gt;
	계통운영 &gt;주요사업 &gt; 전력거래소
  </title>
  <meta charset="utf-8"/>
  <!-- 
	<meta http-equiv="Expires" content="-1" /> 
	<meta http-equiv="Pragma" content="no-cache" /> 
	<meta http-equiv="Cache-Control" content="No-Cache" />
	 -->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="off" name="autocomplete"/>
  <link href="/site/www/css/import.css" rel="stylesheet" type="text/css"/>
  <link href="/site/www/css/tcal.css" rel="stylesheet" type="text/css"/>
  <script src="/site/www/js/jquery-1.9.1.min.js" type="text/javascript">
  </script>
  <script src="/site/www/js/placeholders.min.js" type="text/javascript">
  </script>
  <script src="/site/www/js/jquery.bxslider.min.js" type="text/javascript">
  </script>
  <script src="/site/www/js/common.js" type="text/javascript">
  </script>
  <script src="/site/www/js/banner.js" type=

In [4]:
import re
p = re.compile(r"\d")
result = p.findall("life is too short")
if result is None:
    print(result)

[]
